In [1]:
''' В тестировании стратегии получаем анализ DebtToAssetsпо заданым параметрам
    + выводим итог за нужный нам период по набору тикеров
    + может добавить колонки из key_metrics и income_statment
    +++ добавлена возможность анализировать мин и макс рынка за период сигнала (год)
    
    ! Важно добавить рекомендованный год, ниже которого в таблицу не попадает (сейчас 2022)
'''

' В тестировании стратегии получаем анализ DebtToAssetsпо заданым параметрам\n    + выводим итог за нужный нам период по набору тикеров\n    + может добавить колонки из key_metrics и income_statment\n    +++ добавлена возможность анализировать мин и макс рынка за период сигнала (год)\n    \n    ! Важно добавить рекомендованный год, ниже которого в таблицу не попадает (сейчас 2022)\n'

In [2]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statistics as st

%matplotlib inline 

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a' #job.chap@icloud.com
api_key = 'c81352430e2fe3c941faf0814227562b' #jobs.chaps@gmail.com
# api_key = 'a765d11740cccfb61177da0ad8699d1e' #job.chap@gmail.com

tickers

In [5]:
''' Колонки '''
ticker = 'AAPL'
# ticker = 'AAPL'

# tickers_list = ['AXP', 'INTC']
# tickers_list = ['ADBE', 'AAPL', 'F']
# tickers_list = ['WBA', 'IRBT', 'WRK', 'CSCO', 'KR']
# tickers_list = ['WBA', 'IRBT', 'WRK', 'CSCO', 'KR', 'ADBE', 'AAPL', 'F']
# tickers_list = ['UAA', 'HBAN', 'HPE', 'IVZ', 'KEY', 'RF', 'BEN', 'WMB', 'DAL', 'GM', 'WRK', 'WFC', 'EBAY', 'VTR', 'AIG', 'MU', 'MET', 'NTAP', 'SYY', 'BXP', 'MNST', 'XOM', 'PYPL', 'DIS', 'TROW', 'HLT', 'QCOM', 'CVX', 'AAPL', 'HON', 'UPS', 'LOW', 'ACN', 'SNPS', 'LRCX', 'BKNG']

tickers_list = ['HUBB', 'ZIM', 'GVA', 'R', 'KEX', 'SNA', 'CSGP', 'GXO', 'ABM', 'WSO', 'CAT', 'MIDD', 'LYFT', 'SPR', 'SXI', 'PLXS', 'TT', 'UPS', 'FDX', 'CNI', 'SITE', 'HA', 'CSWI', 'UNP', 'JBHT', 'ODFL', 'AXON', 'CTAS', 'TKR', 'HCSG', 'NSP', 'LMT', 'VVX', 'BLDR', 'AJRD', 'PRLB', 'CHRW', 'BBSI', 'LII', 'AIR', 'BWXT']


data

In [6]:
required_year = '2022'


''' время '''
stime='2010-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [7]:
''' добавляем цифру к году, чтобы видень данные как начало года, а не как факт '''
def change_index_plus_one(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        df_index[i] = str(int(df_index[i])+1)
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

analyse DebtToAssets str

In [8]:
def one_stock_analyse(ticker, api_key, required_year):

    ''' запрос к yahoo'''
    def get_data_from_ticker(tick, interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start='1990-01-01', end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    
    df = get_data_from_ticker(ticker)


    ''' функция по изменению индекса в df из yahoo'''
    def change_index(df):
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

    df = change_index(df)

    ''' получаем нужный период key_metrics'''
    def get_key_metricks(ticker, api_key, required_year):
        key_metrics = fa.key_metrics(ticker, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = change_index_plus_one(df_key_metr)
        df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
        return df_key_metr



    df_key_metr = get_key_metricks(ticker, api_key, required_year)

    ''' проверяем минимальный требуемый уровень и наличие следующего года '''
    if df_key_metr.index[0] >= required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
        last_year = True
    else:
        last_year = False


    ''' получаем индекс из таблицы чтобы получить цены по ним'''
    def get_index(df):
        index_years = df.index.to_list()
        return index_years

    index_years = get_index(df_key_metr)


    ''' извлекаем цены '''
    def get_price_from_df(df, list_dates):
        new_list = []
        for i in range(len(list_dates)):
            x = int(list_dates[i])
            # x += 1
            x = str(x)
            try:
                r = df.loc[f'{x}-01-01']
                r = float(r)
                new_list.append(r)
            except:
                new_list.append(0)

        return new_list

    
    stock_year_prices = get_price_from_df(df, index_years)

    ''' добавляем цены к метрикам'''
    df_key_metr[ticker] = list(stock_year_prices)
    df_key_metr.drop('period', axis=1, inplace=True)

    
    ''' добавляем колонки с изменением в процентах к предыдущему году '''
    def change_percent_all_columns(df):
        x = df.columns.to_list()
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)
            df[f'change % {x[i]}'] = l
        return df

    df_with_changes = change_percent_all_columns(df_key_metr)
    df_with_changes = df_with_changes.reindex(index=df_with_changes.index[::-1])


    df_some_columns = df_with_changes[[f'change % {ticker}', 'change % debtToAssets']]


    # mean_result = st.mean(df_some_columns[f'change % {ticker}'].to_list()[1:])   
    
    
    ''' меняем индекс'''
    df_with_changes.index.name='Date'
    df_with_changes = df_with_changes.reset_index()


    ''' стратегия и сигналы '''
    l = []
    for i in range(len(df_some_columns['change % debtToAssets'])):
        if i <= 1:
            pass
        elif i == len(df_some_columns['change % debtToAssets']) - 1:
            pass
        elif df_with_changes[f'change % {ticker}'][i-1] < df_with_changes[f'change % {ticker}'][i-2] and df_with_changes['change % debtToAssets'][i-1] > df_with_changes['change % debtToAssets'][i-2] and df_with_changes['change % debtToAssets'][i-1] > df_with_changes[f'change % {ticker}'][i-2] and df_with_changes[f'change % {ticker}'][i] < 0:
            l.append([ticker, df_with_changes['Date'][i], df_with_changes[f'change % {ticker}'][i+1]])
        # elif df_with_changes[f'change % {ticker}'][i] < df_with_changes['change % debtToAssets'][i] and df_with_changes[f'change % {ticker}'][i-1] < df_with_changes['change % debtToAssets'][i-1]:
        #     l.append([ticker, df_with_changes['Date'][i], df_with_changes[f'change % {ticker}'][i+1]])
        # elif df_with_changes[f'change % {ticker}'][i] < df_with_changes['change % debtToAssets'][i]:
        #     l.append([ticker, df_with_changes['Date'][i], df_with_changes[f'change % {ticker}'][i+1]])
        else:
            pass  
    
    df_means_results = pd.DataFrame(l, columns = ['Ticker', 'Date', 'Result'])
    print(ticker)

    return df_means_results

In [9]:
# one_stock_analyse(ticker, api_key, required_year)

list analyse

In [10]:
def list_stock_analyse(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        try:
            if i == 0:
                df = one_stock_analyse(tickers_list[i], api_key, required_year)
            else:
                df = pd.concat([df, one_stock_analyse(tickers_list[i], api_key, required_year)], ignore_index=True)
        except:
            pass
    return df


df_means_results = list_stock_analyse(tickers_list, api_key, required_year)

HUBB
ZIM
GVA
R
KEX
SNA
CSGP
GXO
ABM
WSO
CAT
MIDD
LYFT
SPR
SXI
PLXS
TT
FDX
CNI
SITE
HA
CSWI
UNP
JBHT
ODFL
AXON
CTAS
TKR
HCSG
NSP
LMT
VVX
BLDR
AJRD
PRLB
CHRW
BBSI
LII
AIR
BWXT


In [11]:
df_means_results

,Ticker,Date,Result
0,CSGP,2002,-13.333331
1,CSGP,2009,36.326815
2,CAT,2016,60.051298
3,MIDD,2020,65.121247
4,SXI,2009,52.408036
5,CTAS,2008,-29.713038
6,TKR,2016,72.454020
7,NSP,2009,10.815611
8,LMT,2010,10.710935
9,BLDR,2009,172.357712


add additional key metrics

In [12]:
# def change_percent_all_columns(df):
#     x = df.columns.to_list()
#     for i in range(len(x)):
#         v = df[x[i]].to_list()
#         l = []
#         for y in range(len(v)):
#             try:
#                 if y != len(v):
#                     if v[y] < 0 and v[y+1] > 0:
#                         l.append('Minus')
#                     elif v[y] > 0 and v[y+1] < 0:
#                         l.append('Plus')
#                     else:
#                         vv = (v[y] / (v[y+1] / 100)) - 100
#                         l.append(vv) 
#                 else:
#                     vv = 0
#                     l.append(vv)
#                     # print(vv)
#             except:
#                 l.append(0)
#         df[f'change % {x[i]}'] = l
#     return df

In [13]:
# def add_key_metrics(df):
#     v = 'revenuePerShare'
#     v1 = 'netIncomePerShare'
#     v2 = 'inventoryTurnover'
#     v3 = 'capexPerShare'
#     v4 = 'cashPerShare'
#     v5 = 'bookValuePerShare'
#     v6 = 'daysPayablesOutstanding'
#     v7 = 'receivablesTurnover'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             date = df['Date'][i]
#             l.append(key_metrics_change.loc[date, f'change % {v}'])
#             l1.append(key_metrics_change.loc[date,f'change % {v1}'])
#             l2.append(key_metrics_change.loc[date,f'change % {v2}'])
#             l3.append(key_metrics_change.loc[date,f'change % {v3}'])
#             l4.append(key_metrics_change.loc[date,f'change % {v4}'])
#             l5.append(key_metrics_change.loc[date,f'change % {v5}'])
#             l6.append(key_metrics_change.loc[date,f'change % {v6}'])
#             l7.append(key_metrics_change.loc[date,f'change % {v7}'])
#         elif df['Ticker'][i] == df['Ticker'][i-1]:
#             date = df['Date'][i]
#             l.append(key_metrics_change.loc[date, f'change % {v}'])
#             l1.append(key_metrics_change.loc[date,f'change % {v1}'])
#             l2.append(key_metrics_change.loc[date,f'change % {v2}'])
#             l3.append(key_metrics_change.loc[date,f'change % {v3}'])
#             l4.append(key_metrics_change.loc[date,f'change % {v4}'])
#             l5.append(key_metrics_change.loc[date,f'change % {v5}'])
#             l6.append(key_metrics_change.loc[date,f'change % {v6}'])
#             l7.append(key_metrics_change.loc[date,f'change % {v7}'])
#         else:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             date = df['Date'][i]
#             l.append(key_metrics_change.loc[date, f'change % {v}'])
#             l1.append(key_metrics_change.loc[date,f'change % {v1}'])
#             l2.append(key_metrics_change.loc[date,f'change % {v2}'])
#             l3.append(key_metrics_change.loc[date,f'change % {v3}'])
#             l4.append(key_metrics_change.loc[date,f'change % {v4}'])
#             l5.append(key_metrics_change.loc[date,f'change % {v5}'])
#             l6.append(key_metrics_change.loc[date,f'change % {v6}'])
#             l7.append(key_metrics_change.loc[date,f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_key_metrics(df_means_results)

In [14]:
# df_with_parameters.sort_values(by=['Result'], ascending=False)

math (get market mean)

In [15]:
''' get market result to compare '''
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df_market = get_data_from_ticker('^GSPC', stime, None, period)
df_market = df_market.reindex(index=df_market.index[::-1])

def change_index(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        t = df_index[i].date()
        df_index[i] = t.strftime('%Y-%m-%d')
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

df_market = change_index(df_market)

def get_price_from_df(df, stime, ftime):
    new_list = []
    for i in range(int(stime[:4]),int(ftime[:4])):
        x = i
        x += 1
        x = str(x)
        r = df.loc[f'{x}-01-01']
        r = float(r)
        new_list.append(r)
        
    return new_list

df_market_year_velues = get_price_from_df(df_market, stime, ftime)

df_market_year_percents = []
for i in range(len(df_market_year_velues)):
    if i == 0:
        pass
    # elif i == len(df_market_year_velues):
    #     pass
    else:
        df_market_year_percents.append((df_market_year_velues[i] / (df_market_year_velues[i-1]/100))-100)

# st.mean(df_market_year_percents)      



math result

In [16]:
st.mean(df_market_year_percents) 

10.64730018538931

In [17]:
df_means_results['Result'].mean()

42.9267467046635